In [ ]:
import networkx as nx
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy

import networkx.algorithms.community as nx_comm
import random

`pip install markov_clustering[drawing]`

https://markov-clustering.readthedocs.io/en/latest/readme.html

In [ ]:
import markov_clustering as mc

# Community Detection
Partition the graph into subsets (communities) of nodes that more connected to each other than they are with nodes from other subsets.

There are 2 main methods of detecting communities:
1. #### Divisive Methods (Top-down):
    - Start with connected graph
    - Remove edges one by one based on 'betweeness' of nodes, i.e. remove edges between weakly linked nodes first
2. #### Agglomerative Methods (Bottom-Up):
    - Start with a graph of singleton nodes (no edges)
    - Add edges one by one based on 'closeness' of nodes, i.e. join more connected nodes first.


## Edge-Betweenness (Divisive)
Edge-betweenness of an edge is the number of geodesics (shortest paths) that use that edge.
##### Girvan-Newman Algorithm
1. Compute betweenness scores for all edges in the network
2. Remove the edge with the highest score
3. Re-compute betweenness scores for remaining edges 
4. Repeat from 2 until graph is split into 2 (or more) subgraphs

In [ ]:
def edge_to_remove(graph):
    # find edge-betweenness scores for the graph
    G_dict = nx.edge_betweenness_centrality(graph)
    edge = ()

    # extract the edge with highest edge betweenness centrality score
    pairs = sorted(G_dict.items(), key=lambda item: item[1], reverse = True)
    for key, value in pairs:
        edge = key
        break

    return(edge)

In [ ]:
def girvan_newman(graph):
    # find number of connected components
    subgraphs = nx.connected_components(graph)
    subgraph_count = nx.number_connected_components(graph)

    while(subgraph_count == 1): # repeat below until there is more than 1
        source, target = edge_to_remove(graph)
        graph.remove_edge(source,target)
        subgraphs = nx.connected_components(graph)
        subgraph_count = nx.number_connected_components(graph)

    return(subgraphs)

In [ ]:
# Karate club example
G = nx.karate_club_graph()
positions = nx.spring_layout(G)

nx.draw(G, with_labels = True)

In [ ]:
# find communities in the graph
components = girvan_newman(G.copy())

# find the nodes forming the communities
node_groups = []

for i in components:
    node_groups.append(list(i))

In [ ]:
for group in node_groups:
    print(group)

In [ ]:
# plot the communities
color_map = []
for node in G:
    if node in node_groups[0]:
        color_map.append('blue')
    else: 
        color_map.append('green')  

nx.draw(G, node_color=color_map, with_labels=True)

In [ ]:
more_node_groups = []
for group in node_groups:
    # get the sub_graph of the node group
    sub_graph = G.subgraph(group)
    # find the communities in this subgraph
    sub_components = girvan_newman(sub_graph.copy())

    # find the nodes forming the communities
    for i in sub_components:
        more_node_groups.append(list(i))

In [ ]:
for group in more_node_groups:
    print(group)

In [ ]:
# plot the communities
color_map = []
colors = ['blue','cyan','green','lime','purple']
groups = np.arange(len(more_node_groups))
for node in G:
    # identify which group the node is in
    idx = [(node in group) for group in more_node_groups]
    color_map.append(colors[groups[idx][0]]) 

nx.draw(G, node_color=color_map, with_labels=True)
#plt.show()

Can calculate the modularity of the partition after each round of splits to decide when to stop splitting. Generally, stop once further subdivision of groups doesn't lead to increase in modularity.

In [ ]:
print('Modularity of first division:  ' + str(nx_comm.modularity(G,node_groups)))

print('Modularity of second division:  ' + str(nx_comm.modularity(G,more_node_groups)))

## Hierarchical Clustering (Agglomerative)

1. Compute "similarity" of nodes in the graph
    - a variety of ways to define 'similarity'
    - can be based on additional information about the nodes other than connections
    - can use Cosine similarity (can work also for weighted graphs) 
    $$ \sigma_{i j} = \frac{\sum_k A_{i k} A_{k j}}{\sqrt{\sum_k A_{i k}^2 \sum_k A_{j k}^2}} = \frac{n_{i j}}{\sqrt{k_i k_j}}$$
2. Group nodes by how similar they are starting with the most similar pair of nodes

In [ ]:
N = len(G)
# compute similarity matrix
S = np.zeros((N,N))
for i in range(N):
    for j in range(i+1,N):
        nij = len(set(G.neighbors(i)) & set(G.neighbors(j)))
        ki = G.degree(i)
        kj = G.degree(j)
        S[i,j] = nij / np.sqrt(ki*kj)
        S[j,i] = S[i,j]

In [ ]:
Z = hierarchy.linkage((1-S)[np.triu_indices(N,1)],method='average')

Linkage methods: Determine a node's distance from a community
1. Single: The closest node
2. Complete: The furthest node
3. Average: The average distance
 
Use `help(hierarchy.linkage)` to view more linkage methods

In [ ]:
plt.figure(figsize=(16,5))
dn = hierarchy.dendrogram(Z)

In [ ]:
# extract clusters
num_clusters = 2 #number of clusters
cutree = hierarchy.cut_tree(Z, n_clusters=num_clusters).ravel()

In [ ]:
np.c_[range(N),cutree]

In [ ]:
# plot the communities
color_map = []
colors = ['blue','cyan','green','lime','purple','red','orange','yellow']
groups = np.arange(len(more_node_groups))
for node in G:
    # identify which group the node is in
    #idx = [(node in group) for group in sub_node_groups]
    color_map.append(colors[cutree[node]]) 

nx.draw(G, node_color=color_map,pos = positions, with_labels=True)
#plt.show()

## Markov Clustering (MCL)
An unsupervised method (does require choosing the number of communities).

Works by simulating a Markov process (Random walk) in a graph until it reaches equilibrium.
1. Add self-loops to weight matrix $W$ to make it aperiodic, e.g. set $W_{i i}=1$
2. Make a transition probability matrix $P$ from the graph weight matrix by normalizing the rows, i.e. divide by row sums.
3. The random walk is simulated by two alternating operations
    - Expansion: Allows random walk to taker higher length paths between nodes. Expansion parameter $n \geq 2$
    $$ P \rightarrow P^n$$
    - Inflation: Changes transition probability matrix to favour more probable walks. Inflation parameter $r>1$ $$ P_{i j} \rightarrow \frac{P_{i j}^r}{\sum_k P_{i k}^r}$$
4. Should converge to a disconnected P defining communities by absorbing components.

In [ ]:
# Build adjacency matrix
A = nx.to_numpy_matrix(G)

In [ ]:
# Run MCL algorithm
n = 2  #expansion parameter
r = 2  #inflation parameter
result = mc.run_mcl(A, expansion = n, inflation = r)
clusters = mc.get_clusters(result)

# Draw clusters
mc.draw_graph(A, clusters, pos = positions,with_labels=True, edge_color="silver")

In [ ]:
for group in clusters:
    print(group)

In [ ]:
for group in node_groups:
    print(group)